#Chart2Go 


In [22]:
#!/usr/bin/env python3
"""
NQ Futures Dashboard Generator - CSS CANDLESTICK VERSION
Generates an HTML page with CSS candlestick chart and dashboard data for NQ futures
"""

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import os

def get_nq_data():
    """Fetch NQ futures data from Yahoo Finance"""
    ticker = "NQ=F"
    
    try:
        ticker_obj = yf.Ticker(ticker)
        daily_data = ticker_obj.history(period="1mo", interval="1d")
        return daily_data
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def create_sample_data():
    """Create sample NQ futures data"""
    import numpy as np
    
    dates = pd.date_range(start='2024-05-01', end='2024-06-10', freq='D')
    np.random.seed(42)
    
    base_price = 18000
    price_data = []
    current_price = base_price
    
    for i in range(len(dates)):
        daily_change = np.random.normal(10, 100)
        current_price += daily_change
        
        open_price = current_price
        high_price = open_price + abs(np.random.normal(0, 50))
        low_price = open_price - abs(np.random.normal(0, 50))
        close_price = low_price + (high_price - low_price) * np.random.random()
        
        price_data.append({
            'Open': open_price,
            'High': high_price,
            'Low': low_price,
            'Close': close_price,
            'Volume': int(np.random.normal(75000, 15000))
        })
        
        current_price = close_price
    
    return pd.DataFrame(price_data, index=dates)

def calculate_range(data, periods):
    """Calculate average range for given periods"""
    if data is None or len(data) < periods:
        return 0.0
    
    recent_data = data.tail(periods)
    ranges = recent_data['High'] - recent_data['Low']
    return float(round(ranges.mean(), 2))

def create_candlestick_html(data):
    """Create CSS candlestick chart HTML"""
    # Get last 20 days for better display
    chart_data = data.tail(20)
    
    # Calculate price range for scaling
    min_price = chart_data[['Open', 'High', 'Low', 'Close']].min().min()
    max_price = chart_data[['Open', 'High', 'Low', 'Close']].max().max()
    price_range = max_price - min_price
    
    candlesticks_html = ""
    
    for i, (date, row) in enumerate(chart_data.iterrows()):
        open_price = row['Open']
        high_price = row['High']
        low_price = row['Low']
        close_price = row['Close']
        
        # Determine if bullish (green) or bearish (red)
        is_bullish = close_price >= open_price
        color = "#00ff88" if is_bullish else "#ff4040"
        
        # Calculate positions (inverted because CSS top:0 is at top)
        high_pos = ((max_price - high_price) / price_range) * 100
        low_pos = ((max_price - low_price) / price_range) * 100
        
        if is_bullish:
            body_top = ((max_price - close_price) / price_range) * 100
            body_bottom = ((max_price - open_price) / price_range) * 100
        else:
            body_top = ((max_price - open_price) / price_range) * 100
            body_bottom = ((max_price - close_price) / price_range) * 100
        
        body_height = body_bottom - body_top
        if body_height < 1:  # Ensure minimum visibility for doji candles
            body_height = 1
        
        # Create candlestick HTML
        candlesticks_html += f'''
            <div class="candlestick" style="left: {i * 4 + 2}%;">
                <div class="wick" style="top: {high_pos}%; height: {low_pos - high_pos}%;"></div>
                <div class="body" style="top: {body_top}%; height: {body_height}%; background-color: {color};"></div>
                <div class="date-label">{date.strftime('%m/%d')}</div>
                <div class="price-tooltip">
                    O: ${open_price:.0f}<br>
                    H: ${high_price:.0f}<br>
                    L: ${low_price:.0f}<br>
                    C: ${close_price:.0f}
                </div>
            </div>
        '''
    
    return candlesticks_html

def generate_html():
    """Generate the HTML dashboard"""
    
    # Get data
    data = get_nq_data()
    
    if data is None:
        print("Failed to fetch data. Using sample data.")
        data = create_sample_data()
    
    # Calculate metrics
    current_price = float(round(data['Close'].iloc[-1], 2))
    current_volume = int(data['Volume'].iloc[-1])
    day_range = calculate_range(data, 1)
    week_range = calculate_range(data, 7)
    hour_range = calculate_range(data, 1)
    min15_range = calculate_range(data, 1)
    
    # Create candlestick chart
    candlesticks_html = create_candlestick_html(data)
    
    last_updated = datetime.now().strftime('%Y-%m-%d %H:%M:%S EST')
    
    html = f'''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>NQ Futures Dashboard</title>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            background: linear-gradient(135deg, #0c0c0c 0%, #1a1a1a 100%);
            color: #e0e0e0;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            min-height: 100vh;
            padding: 20px;
        }}
        
        .container {{
            max-width: 1400px;
            margin: 0 auto;
        }}
        
        .header {{
            text-align: center;
            margin-bottom: 30px;
            padding: 20px 0;
            border-bottom: 2px solid #333;
        }}
        
        .header h1 {{
            font-size: 2.5em;
            color: #00ff88;
            text-shadow: 0 0 10px rgba(0, 255, 136, 0.3);
            margin-bottom: 10px;
        }}
        
        .subtitle {{
            font-size: 1.1em;
            color: #888;
        }}
        
        .dashboard {{
            display: grid;
            grid-template-columns: 1fr 350px;
            gap: 20px;
            margin-bottom: 20px;
        }}
        
        .chart-container {{
            background: rgba(20, 20, 20, 0.8);
            border-radius: 12px;
            padding: 20px;
            border: 1px solid #333;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
        }}
        
        .chart-title {{
            color: #00ff88;
            font-size: 1.2em;
            margin-bottom: 15px;
            text-align: center;
        }}
        
        .chart-wrapper {{
            position: relative;
            height: 400px;
            background: rgba(10, 10, 10, 0.5);
            border: 1px solid #333;
            border-radius: 8px;
            overflow: hidden;
        }}
        
        .candlestick {{
            position: absolute;
            width: 3%;
            height: 100%;
        }}
        
        .wick {{
            position: absolute;
            left: 50%;
            transform: translateX(-50%);
            width: 1px;
            background-color: #888;
        }}
        
        .body {{
            position: absolute;
            left: 20%;
            width: 60%;
            border: 1px solid rgba(255, 255, 255, 0.3);
        }}
        
        .candlestick:hover .price-tooltip {{
            display: block;
        }}
        
        .price-tooltip {{
            display: none;
            position: absolute;
            top: -60px;
            left: 50%;
            transform: translateX(-50%);
            background: rgba(0, 0, 0, 0.9);
            color: #fff;
            padding: 8px;
            border-radius: 4px;
            font-size: 0.8em;
            white-space: nowrap;
            z-index: 100;
            border: 1px solid #333;
        }}
        
        .date-label {{
            position: absolute;
            bottom: -25px;
            left: 50%;
            transform: translateX(-50%);
            font-size: 0.7em;
            color: #666;
            white-space: nowrap;
        }}
        
        .stats-panel {{
            background: rgba(20, 20, 20, 0.8);
            border-radius: 12px;
            padding: 20px;
            border: 1px solid #333;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
        }}
        
        .stat-card {{
            background: rgba(30, 30, 30, 0.6);
            border-radius: 8px;
            padding: 15px;
            margin-bottom: 15px;
            border-left: 4px solid #00ff88;
            transition: all 0.3s ease;
        }}
        
        .stat-card:hover {{
            background: rgba(40, 40, 40, 0.8);
            transform: translateX(5px);
        }}
        
        .stat-label {{
            font-size: 0.9em;
            color: #aaa;
            margin-bottom: 5px;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }}
        
        .stat-value {{
            font-size: 1.4em;
            font-weight: bold;
            color: #fff;
        }}
        
        .price {{
            color: #00ff88;
            font-size: 1.6em !important;
        }}
        
        .volume {{
            color: #4da6ff;
        }}
        
        .range {{
            color: #ffaa00;
        }}
        
        .footer {{
            text-align: center;
            margin-top: 30px;
            padding-top: 20px;
            border-top: 1px solid #333;
            color: #666;
            font-size: 0.9em;
        }}
        
        .legend {{
            margin-top: 10px;
            text-align: center;
            font-size: 0.9em;
        }}
        
        .legend-item {{
            display: inline-block;
            margin: 0 15px;
            color: #888;
        }}
        
        .legend-color {{
            display: inline-block;
            width: 12px;
            height: 12px;
            margin-right: 5px;
            vertical-align: middle;
        }}
        
        .bullish {{
            background-color: #00ff88;
        }}
        
        .bearish {{
            background-color: #ff4040;
        }}
        
        @media (max-width: 1200px) {{
            .dashboard {{
                grid-template-columns: 1fr;
            }}
            
            .stats-panel {{
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                gap: 15px;
            }}
            
            .stat-card {{
                margin-bottom: 0;
            }}
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>NQ FUTURES DASHBOARD</h1>
            <div class="subtitle">NASDAQ 100 E-mini Futures • Candlestick Chart</div>
        </div>
        
        <div class="dashboard">
            <div class="chart-container">
                <div class="chart-title">NQ Futures - Last 20 Trading Days</div>
                <div class="chart-wrapper">
                    {candlesticks_html}
                </div>
                <div class="legend">
                    <div class="legend-item">
                        <span class="legend-color bullish"></span>Bullish (Close ≥ Open)
                    </div>
                    <div class="legend-item">
                        <span class="legend-color bearish"></span>Bearish (Close < Open)
                    </div>
                </div>
            </div>
            
            <div class="stats-panel">
                <div class="stat-card">
                    <div class="stat-label">Current Price</div>
                    <div class="stat-value price">${current_price:,.2f}</div>
                </div>
                
                <div class="stat-card">
                    <div class="stat-label">Volume</div>
                    <div class="stat-value volume">{current_volume:,}</div>
                </div>
                
                <div class="stat-card">
                    <div class="stat-label">Day Range (Avg)</div>
                    <div class="stat-value range">{day_range} pts</div>
                </div>
                
                <div class="stat-card">
                    <div class="stat-label">Week Range (Avg)</div>
                    <div class="stat-value range">{week_range} pts</div>
                </div>
                
                <div class="stat-card">
                    <div class="stat-label">Hour Range (Avg)</div>
                    <div class="stat-value range">{hour_range} pts</div>
                </div>
                
                <div class="stat-card">
                    <div class="stat-label">15min Range (Avg)</div>
                    <div class="stat-value range">{min15_range} pts</div>
                </div>
            </div>
        </div>
        
        <div class="footer">
            <p>Last Updated: {last_updated} | Data provided by Yahoo Finance</p>
            <p>⚠️ This is for educational purposes only. Not financial advice.</p>
            <p>💡 Hover over candlesticks to see OHLC data</p>
        </div>
    </div>
</body>
</html>'''
    
    return html

def main():
    """Main function"""
    print("Generating NQ Futures Dashboard...")
    
    html_content = generate_html()
    
    filename = f"nq_dashboard_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    full_path = os.path.abspath(filename)
    file_url = f"file://{full_path}"
    
    print(f"\n✅ Dashboard generated successfully!")
    print(f"📁 File location: {full_path}")
    print(f"🔗 Click to open: {file_url}")
    print(f"\nOr copy and paste this link into your browser:")
    print(f"{file_url}")
    print("\n" + "="*60)

if __name__ == "__main__":
    try:
        import yfinance
        import pandas
        import numpy
    except ImportError:
        print("Installing required packages...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "yfinance", "pandas", "numpy"])
    
    main()

Generating NQ Futures Dashboard...
Error fetching data: Too Many Requests. Rate limited. Try after a while.
Failed to fetch data. Using sample data.

✅ Dashboard generated successfully!
📁 File location: c:\Users\Wolfrank\Desktop\DDesktop\CodeWolf\QuantTrading-1\Tools\Chart2go\nq_dashboard_20250610_154010.html
🔗 Click to open: file://c:\Users\Wolfrank\Desktop\DDesktop\CodeWolf\QuantTrading-1\Tools\Chart2go\nq_dashboard_20250610_154010.html

Or copy and paste this link into your browser:
file://c:\Users\Wolfrank\Desktop\DDesktop\CodeWolf\QuantTrading-1\Tools\Chart2go\nq_dashboard_20250610_154010.html

